## Leer data de una Base de Datos

### Cargo Paquetes Iniciales

In [1]:
from functions.create_db import load_data, dict_Empresas, write_sql_main
import pandas as pd
import sqlite3

### Cargo Datos (Clase Anterior)

In [2]:
df = load_data(dict_Empresas)

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Archivo Apple cargado exitosamente.


[*********************100%***********************]  1 of 1 completed


Archivo Microsoft cargado exitosamente.


[*********************100%***********************]  1 of 1 completed


Archivo Meta cargado exitosamente.


[*********************100%***********************]  1 of 1 completed

Archivo Tesla cargado exitosamente.


In [3]:
df

,Date,Close,High,Low,Open,Volume,Empresa,TKT
0,2023-01-03,123.470619,129.226060,122.582127,128.613993,112117500,Apple,AAPL
1,2023-01-04,124.744125,127.014716,123.480495,125.267347,89113600,Apple,AAPL
2,2023-01-05,123.421249,126.136083,123.164580,125.504267,80962700,Apple,AAPL
3,2023-01-06,127.962433,128.623863,123.292924,124.398604,87754700,Apple,AAPL
4,2023-01-09,128.485687,131.704009,128.229017,128.801602,70790800,Apple,AAPL
...,...,...,...,...,...,...,...,...
590,2025-05-12,318.380005,322.209991,311.500000,321.989990,112826700,Tesla,TSLA
591,2025-05-13,334.070007,337.589996,316.799988,320.000000,136992600,Tesla,TSLA
592,2025-05-14,347.679993,350.000000,337.000000,342.500000,136997300,Tesla,TSLA
593,2025-05-15,342.820007,346.140015,334.720001,340.339996,97882600,Tesla,TSLA


In [4]:
write_sql_main(DB='../local/database/stock_db.db',
               df = df,
               table="main")

Connected to SQLite
The SQLite connection is closed


'ok'

### Leer Info de una Base de Datos

In [9]:
#Me conecto a la Base de Datos
conn = sqlite3.connect('../local/database/stock_db.db')

In [10]:
#¿Como ver las tablas de una BD?
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table'", conn)
tables.head()

,name
0,main


In [11]:
#¿Como ver las tabla de compra_ventas?
main_table = pd.read_sql_query("SELECT * FROM main", conn)
main_table.head()

,Date,Close,High,Low,Open,Volume,Empresa,TKT
0,2023-01-03 00:00:00,123.470619,129.226060,122.582127,128.613993,112117500,Apple,AAPL
1,2023-01-04 00:00:00,124.744125,127.014716,123.480495,125.267347,89113600,Apple,AAPL
2,2023-01-05 00:00:00,123.421249,126.136083,123.164580,125.504267,80962700,Apple,AAPL
3,2023-01-06 00:00:00,127.962433,128.623863,123.292924,124.398604,87754700,Apple,AAPL
4,2023-01-09 00:00:00,128.485687,131.704009,128.229017,128.801602,70790800,Apple,AAPL


In [12]:
conn.close()

## Export to csv

In [13]:
main_table.to_csv('../local/csvs/stock_db.csv')

## Conexión con BigQuery

In [8]:
# Generamos conexion a BQ
from google.oauth2 import service_account # para generar conexion con BigQuery
#bq_cred = service_account.Credentials.from_service_account_file('../local/tokens/gcp_token2.json')
bq_cred = service_account.Credentials.from_service_account_file('../local/tokens/gcp_token_clase.json')

bq_cred

In [7]:
# Vemos que hay en "main_table"
main_table.head()

,index,Date,Open,High,Low,Close,Adj Close,Volume,Empresa
0,0,2023-01-03,130.279999,130.899994,124.169998,125.070000,124.879326,112117500,Apple
1,1,2023-01-04,126.889999,128.660004,125.080002,126.360001,126.167366,89113600,Apple
2,2,2023-01-05,127.129997,127.769997,124.760002,125.019997,124.829399,80962700,Apple
3,3,2023-01-06,126.010002,130.289993,124.889999,129.619995,129.422394,87754700,Apple
4,4,2023-01-09,130.470001,133.410004,129.889999,130.149994,129.951584,70790800,Apple


### Cargar Datos en BQ

In [10]:
# Enviamos el df a BQ - Podemos crear o reemplazar una tabla
import pandas_gbq # para interactuar con BQ

pandas_gbq.to_gbq(main_table, 'Test.clase2024', project_id= 'fantina-v1',
                   if_exists= 'replace', credentials = bq_cred)

In [19]:
# Si queremos anexar datos de una tabla en lugar de reemplazar
pandas_gbq.to_gbq(main_table, 'Test.clase2024', project_id= 'fantina-v1',
                   if_exists= 'append', credentials = bq_cred)

In [11]:
# Renombramos la columna "Adj Close" a "Adj_Close"
main_table = main_table.rename(columns={'Adj Close': 'Adj_Close'})

pandas_gbq.to_gbq(main_table, 'Test.clase2024_v2', project_id= 'fantina-v1',
                   if_exists= 'replace', credentials = bq_cred)

### Leer Datos de BQ

In [20]:
# Si queremos leer una tabla
sql = """SELECT * FROM `fantina-v1.Test.clase2024`"""
df_bq = pd.read_gbq(sql, project_id='fantina-v1', credentials = bq_cred, dialect='standard') 

df_bq.head()

/var/folders/84/4j54nkt90_l921kzy5yzj1g00000gn/T/ipykernel_97388/652018203.py:3: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_bq = pd.read_gbq(sql, project_id='fantina-v1', credentials = bq_cred, dialect='standard')


,index,Date,Open,High,Low,Close,Adj Close,Volume,Empresa
0,0,2023-01-03,130.279999,130.899994,124.169998,125.070000,124.048050,112117500,Apple
1,1,2023-01-04,126.889999,128.660004,125.080002,126.360001,125.327507,89113600,Apple
2,2,2023-01-05,127.129997,127.769997,124.760002,125.019997,123.998459,80962700,Apple
3,3,2023-01-06,126.010002,130.289993,124.889999,129.619995,128.560867,87754700,Apple
4,4,2023-01-09,130.470001,133.410004,129.889999,130.149994,129.086533,70790800,Apple
